In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/qa_pairs_filtered.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from datasets import DatasetDict

# Veri kümenizin %15'ini alarak yeni bir veri kümesi oluşturun
fraction = 0.15
train_dataset = dataset['train']

# Yeni veri kümesi için %15'lik bir örneklem seçin
new_train_dataset = train_dataset.shuffle(seed=42).select(range(int(len(train_dataset) * fraction)))

# Yeni bir DatasetDict oluşturun ve veriyi ekleyin
new_dataset = DatasetDict({
    'train': new_train_dataset
})

# Yeni veri kümesinin boyutunu kontrol edin
print(new_dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 108028
    })
})


In [6]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

df = new_dataset['train'].to_pandas()

# Eğitim ve test setlerine bölün
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# DataFrame'leri tekrar dataset formatına dönüştürün
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [7]:
train_dataset

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 86422
})

In [8]:
test_dataset

Dataset({
    features: ['input', 'output', '__index_level_0__'],
    num_rows: 21606
})

In [9]:
from transformers import Trainer, TrainingArguments, AutoModelForSeq2SeqLM, AutoTokenizer

# Tokenizer ve model yükleme
model_name = "t5-small"  # veya kullanmak istediğiniz başka bir model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

max_input_length = 128
max_target_length = 128

# Veriyi tokenizasyon için hazırlama
def preprocess_function(examples):
    inputs = examples['input']
    targets = examples['output']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding= "max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding= "max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Eğitim argümanları
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir='/content/drive/MyDrive/t5results',
    num_train_epochs=3,
    logging_dir='/content/drive/MyDrive/t5results/logs',
    logging_steps=100,
    load_best_model_at_end=True,
    save_total_limit=5,
    save_steps=500,
    evaluation_strategy="steps",
)

# Trainer oluşturma
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/86422 [00:00<?, ? examples/s]

Map:   0%|          | 0/21606 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:

# # Modeli eğitme
trainer.train()

KeyboardInterrupt: 